In [770]:
import html
from bs4 import BeautifulSoup
import pandas as pd
from unidecode import unidecode
import numpy as np
from sklearn.model_selection import train_test_split
import py_entitymatching as em
from sklearn.linear_model import LogisticRegression
import recordlinkage
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

# Data cleaning:

In [771]:
data_ACM = pd.read_csv(r'ACM.csv')
data_DBLP = pd.read_csv(r'DBLP2_new.csv')
#data_ACM['venue'] = data_ACM['venue'].to_string()

In [772]:
data_ACM['venue'].str.strip()

0                                    International Conference on Management of Data
1                                    International Conference on Management of Data
2                                    International Conference on Management of Data
3                                    International Conference on Management of Data
4                                    International Conference on Management of Data
                                           ...                                     
2289                                                          Very Large Data Bases
2290    The VLDB Journal &mdash; The International Journal on Very Large Data Bases
2291                                                          Very Large Data Bases
2292                                                              ACM SIGMOD Record
2293                                                          Very Large Data Bases
Name: venue, Length: 2294, dtype: object

In [773]:
#data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
#data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [774]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))

In [775]:
#data_ACM['venue'] = data_ACM['venue'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))
data_ACM['venue'] = data_ACM['venue'].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

In [776]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: unidecode(str(x)))
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: unidecode(str(x)))

In [777]:
data_ACM['title'] = data_ACM['title'].apply(lambda x: x.upper())
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: x.upper())

In [778]:
data_DBLP['title'] = data_DBLP['title'].apply(lambda x: x.upper())
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: x.upper())

In [779]:
data_ACM.replace("Very Large Data Bases", "VLDB", inplace=True)
data_ACM.replace("International Conference on Management of Data", "SIGMOD Conference", inplace=True)
data_ACM.replace("ACM SIGMOD Record ", "SIGMOD Record", inplace=True)
data_ACM.replace("ACM Transactions on Database Systems (TODS) ", "ACM Trans. Database Syst. ", inplace=True)
data_ACM.replace("The VLDB Journal — The International Journal on Very Large Data Bases ", "VLDB J.", inplace=True)

In [780]:
#data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
#data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [781]:
data_ACM.to_csv(r'ACM3.csv', encoding='utf-8', index=False)
data_DBLP.to_csv(r'DBLP3.csv', encoding='utf-8', index=False)

In [782]:
#data_ACM['venue'] = data_ACM['venue'].to_string()

In [783]:
#for col in data_ACM.columns:
#   data_ACM[col] = data_ACM[col].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

In [784]:
#data_ACM.title.str.lstrip()
#data_ACM.authors.str.lstrip()
#data_ACM.venue.str.lstrip()
#print(data_ACM.loc[2160, 'venue'])
#print(data_ACM.dtypes)

In [785]:
#data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
#data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [786]:
#d2['authors'] = data_ACM.apply(lambda x : x.split(","))
#for i, row in data_ACM.iterrows():
 #   data_ACM.at[i, 'authors'] = data_ACM.at[i, 'authors'].split(",")
#data_ACM['authors'] = data_ACM['authors'].str.split(", ")
#data_ACM['authors'] = data_ACM['authors'].str.split(" ")

In [787]:
#html_data=unicodedata.normalize('NFKD',table.to_html()).encode('ascii','ignore')

# Blocking:

The approaches in this section didn't work

In [788]:
em.set_key(data_ACM, 'id')
em.set_key(data_DBLP, 'id')
em.to_csv_metadata(data_ACM, r'ACM3_meta.csv')
em.to_csv_metadata(data_ACM, r'DBLP3_meta.csv')

File already exists at ACM3_meta.csv; Overwriting it
Metadata file already exists at ACM3_meta.metadata. Overwriting it
File already exists at DBLP3_meta.csv; Overwriting it
Metadata file already exists at DBLP3_meta.metadata. Overwriting it


True

In [789]:
print(C1)

       _id  ltable_id  rtable_id  \
0        0     304586     304586   
1        1     304587     304587   
2        2     304589     304589   
3        3     304590     304590   
4        4     304582     304582   
...    ...        ...        ...   
2472  2472     950483     950483   
2473  2473     672977     672977   
2474  2474     672980     672980   
2475  2475     945741     945741   
2476  2476     672979     672979   

                                                                                             ltable_title  \
0                                                    THE WASA2 OBJECT-ORIENTED WORKFLOW MANAGEMENT SYSTEM   
1                                 A USER-CENTERED INTERFACE FOR QUERYING DISTRIBUTED MULTIMEDIA DATABASES   
2                                            WORLD WIDE DATABASE-INTEGRATING THE WEB, CORBA AND DATABASES   
3                                                                XML-BASED INFORMATION MEDIATION WITH MIX   
4                     

In [790]:
C1.to_csv(r'C1.csv', encoding='utf-8', index=False)

In [791]:
dfA =C1[['ltable_id', 'ltable_authors', 'ltable_title', 'ltable_venue', 'ltable_year']].copy()
#['ltable_id', 'ltable_authors', 'ltable_title', 'ltable_venue', 'ltable_year']
dfB = C1[['rtable_id', 'rtable_authors', 'rtable_title', 'rtable_venue', 'rtable_year']].copy()
#['rtable_id', 'rtable_authors', 'rtable_title', 'rtable_venue', 'rtable_year']

In [792]:
x = C1.groupby('ltable_title')

In [793]:
print(x)

In [794]:
stacked = pd.concat([data_ACM, data_DBLP], keys= ['ACM', 'DBLP'], ignore_index=True)
#dfA_title = ACM.groupby('title')
#dfB_title = DBLP.groupby('title')

In [795]:
stacked.to_csv(r'stacked.csv', encoding='utf-8', index=False)

In [796]:
grouped = stacked.groupby('title')

In [797]:
print(len(stacked.groupby('title')))

2791


In [798]:
print(len(stacked))

4910


In [799]:
abc = pd.DataFrame(columns=['idDBLP', 'idACM'])

In [800]:
sum=0
grouped.size()
for key, item in grouped:
    g = grouped.get_group(key)
    if g.size == 10:
        abc = abc.append({'idDBLP' : g.get('id').iloc[1], 'idACM' : g.get('id').iloc[0]}, ignore_index=True)
        #print(g.get('id').iloc[1])
        

In [801]:
lev = pd.DataFrame(columns=['idDBLP', 'idACM'])

In [802]:
#compare = em.string_compare_levenshtein(C, 'title', 'title') 
#for key, item in grouped:
 #   g = grouped.get_group(key)
  #  s = g.size
   # if s > 5:
    #    compare = em.string_compare_levenshtein(g, 'authors', 'authors')
     #   if compare >= 0.7:
      #      if s < 10:
       #         lev = lev.append({'idDBLP' : g.get('id').iloc[1], 'idACM' : g.get('id').iloc[0]}, ignore_index=True)
            
        

In [803]:
print(abc.head(10))

                         idDBLP   idACM
0     conf/vldb/HumborstadSHT97  671000
1          conf/vldb/JohnsonS94  672996
2       journals/sigmod/CoxEH96  245901
3       conf/sigmod/LahiriJJC98  276366
4          conf/vldb/DeenJNNW00  671702
5    journals/vldb/VerykiosME03  775455
6  journals/sigmod/LaenderRST02  565137
7      journals/tods/KotidisR01  503100
8    journals/sigmod/KeetonPH98  290602
9           conf/vldb/PantiSG00  671693


In [804]:
abc2 = abc['idACM'].apply(lambda x : str(x))

In [805]:
abc2 = abc['idDBLP'].apply(lambda x : str(x))

In [806]:
#abc2.sort_values('idDBLP')

In [807]:
#abc2.to_csv(r'abc2.csv', encoding='utf-8', index=False)

In [808]:
#abc.to_csv(r'abc.csv', encoding='utf-8', index=False)

In [809]:
#perf = pd.read_csv(r'DBLP-ACM_perfectMapping.csv')

In [810]:
#perf['idDBLP'].nunique()

In [811]:
#perf['idACM'].nunique()

In [812]:
#perf['idDBLP'].count()

# Blocking 2. way

In [813]:
#em.set_key(stacked, 'id')
#em.to_csv_metadata(stacked, r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project/stacked_meta.csv')

In [814]:
#stack2 = em.read_csv_metadata(r'stacked_meta.csv')

In [815]:
#ACM = em.read_csv_metadata(r'ACM3_meta.csv', key='id')
#DBLP = em.read_csv_metadata(r'DBLP3_meta.csv', key='id')

In [816]:
#b1 = em.AttrEquivalenceBlocker() # , 'title', 'authors', 'venue', 'year'

In [817]:
#C1 = b1.block_tables(ACM, DBLP, 'title', 'title', 
 #    l_output_attrs=['title', 'authors', 'venue', 'year'], 
  #   r_output_attrs=['title', 'authors', 'venue', 'year'], allow_missing=True)

In [818]:
indexer = recordlinkage.Index()
indexer.block(['venue', 'year'])
candidate_links = indexer.index(data_ACM, data_DBLP)

In [819]:
#The blocks by index:
print(candidate_links)

MultiIndex([(   0,    5),
            (   0,   30),
            (   0,   65),
            (   0,  101),
            (   0,  127),
            (   0,  166),
            (   0,  230),
            (   0,  232),
            (   0,  269),
            (   0,  323),
            ...
            (2290, 1465),
            (2290, 1562),
            (2290, 1609),
            (2290, 1637),
            (2290, 1686),
            (2290, 1798),
            (2290, 1991),
            (2290, 2135),
            (2290, 2338),
            (2290, 2511)],
           length=156053)


In [820]:
"""compare_cl = recordlinkage.Compare()
compare_cl.exact("venue", "venue", label="venue")
compare_cl.string("title", "title", method="levenshtein", label="title")
compare_cl.exact("year", "year", label="year")
compare_cl.string("authors", "authors", method="levenshtein", label="authors")
features = compare_cl.compute(candidate_links, data_ACM, data_DBLP)
print(features)"""

'compare_cl = recordlinkage.Compare()\ncompare_cl.exact("venue", "venue", label="venue")\ncompare_cl.string("title", "title", method="levenshtein", label="title")\ncompare_cl.exact("year", "year", label="year")\ncompare_cl.string("authors", "authors", method="levenshtein", label="authors")\nfeatures = compare_cl.compute(candidate_links, data_ACM, data_DBLP)\nprint(features)'

In [821]:
#Jaro-Winkler distance between ACM-DBLP venues and authors
compare_cl = recordlinkage.Compare()
compare_cl.exact("venue", "venue", label="venue")
compare_cl.string("title", "title", method="jarowinkler", label="title")
compare_cl.exact("year", "year", label="year")
compare_cl.string("authors", "authors", method="jarowinkler", label="authors")
features = compare_cl.compute(candidate_links, data_ACM, data_DBLP)

In [822]:
features

venue     title  year   authors
0    5         1  0.641764     1  0.545622
     30        1  0.675314     1  0.572156
     65        1  0.663559     1  0.376344
     101       1  0.641602     1  0.534008
     127       1  0.543363     1  0.505615
...          ...       ...   ...       ...
2290 1798      1  0.538269     1  0.558967
     1991      1  0.585738     1  0.720678
     2135      1  0.550294     1  0.596184
     2338      1  0.507937     1  0.639254
     2511      1  0.526685     1  0.619514

[156053 rows x 4 columns]

In [823]:
matches = features[features[['title','authors']].sum(axis=1)>= 1.56] 
#1.56 threshold was determined through trying different values

In [824]:
matches.index.names = ['ACM','DBLP']
matches=matches.reset_index()
matches_new=matches.loc[matches.groupby(['ACM'])['DBLP'].idxmax()]
predictions=matches_new.set_index(['ACM', 'DBLP'])

In [825]:
predictions.to_csv('predictions.csv')

In [826]:
perf_match = pd.read_csv('DBLP-ACM_perfectMapping.csv', header=0, encoding="utf-8")

In [827]:
df1 = data_ACM['id'].to_dict()
df2 = data_DBLP['id'].to_dict()
df1_rev = {y: x for x, y in df1.items()}
df2_rev = {y: x for x, y in df2.items()}
perf_match['idACM'] = perf_match['idACM'].map(df1_rev)
perf_match['idDBLP'] = perf_match['idDBLP'].map(df2_rev)
perf_match = perf_match[['idACM', 'idDBLP']]
print(perf_match)
match_true = pd.MultiIndex.from_frame(perf_match)

      idACM  idDBLP
0       301     143
1       306    1530
2       292     310
3       289    2090
4       308    1744
...     ...     ...
2219    597     781
2220   1736    2004
2221    523    2260
2222    606     178
2223    925    2368

[2224 rows x 2 columns]


In [828]:
f_score=recordlinkage.fscore(match_true, predictions.index)

In [829]:
print(f_score)

0.942464477055672


# Task 2

First we add label 1 = match, 0 = no match 'label'

In [830]:
features['label'] = 0

In [831]:
for i in predictions.index.tolist():
    features.loc[i, 'label'] = 1

In [832]:
features.to_csv('labeled_data.csv')

In [833]:
features

venue     title  year   authors  label
0    5         1  0.641764     1  0.545622      0
     30        1  0.675314     1  0.572156      0
     65        1  0.663559     1  0.376344      0
     101       1  0.641602     1  0.534008      0
     127       1  0.543363     1  0.505615      0
...          ...       ...   ...       ...    ...
2290 1798      1  0.538269     1  0.558967      0
     1991      1  0.585738     1  0.720678      0
     2135      1  0.550294     1  0.596184      0
     2338      1  0.507937     1  0.639254      0
     2511      1  0.526685     1  0.619514      0

[156053 rows x 5 columns]

Then we create the machine learning models:

In [834]:
X_train, X_test, y_train, y_test = train_test_split(features[['title', 'authors', 'venue', 'year']],
                                        features['label'], test_size=0.3, random_state=100)
print(X_train.size, X_test.size, y_train.size, y_test.size)

436948 187264 109237 46816


In [835]:
reg = LogisticRegression()

In [836]:
mod = reg.fit(X_train, y_train)
mod2 = mod.predict(X_test)
result_log = f1_score(y_test, mod2)

In [837]:
print(result_log)

0.9720930232558139


In [838]:
clf = svm.SVC()

In [839]:
clf.fit(X_train, y_train)

SVC()

In [840]:
svm = clf.fit(X_train, y_train)
svm2 = svm.predict(X_test)
result_svm = f1_score(y_test, svm2)

In [841]:
print(result_svm)

0.9777094542659492


In [842]:
print(clf.score(X_test, y_test))

0.9993805536568694


In [ ]:
param_grid = {'C': [1, 5, 10], 'kernel': ['linear', 'poly','rbf'],'degree':[2,3],'gamma':[0.1,1]}
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

In [ ]:
print("best parameters: ", grid_search.best_params_)

In [ ]:
#best = svm.SVC(C=10, degree=3, gamma=1, kernel='poly')